# Topic ideas

---

Group name:  John Anderson Torres Mosquera

---


## Inbound Logistics Forecasting Benchmark

### Data source

The problem at hand refers to forecasting the inbound material volume (in tons) on monthly basis for the next 4 months for an international automotive company. 

The motivation behind that was the lack of synchronization between suppliers and freight forwarders systems, causing over- or under-capacity planning whenever a plant’s material demands change abruptly, leading to higher logistics transportation costs. 

I published a paper last year about this system, as a result of a research on inbound forecasting systems that I started in 2018. It can be found here [Forecasting System for Inbound Logistics Material Flows at an International Automotive Company](https://www.mdpi.com/2673-4591/39/1/75). Howerver this codebase was written in R using the forecasting package [forecast](https://cran.r-project.org/package=forecast) by Robert Hyndman and George Athanasopoulos in their book [Forecasting at Scale](https://otexts.com/fpp3/). It included algorithms like SARIMA, Exponential Smoothing, Multilayer Neural Networks, Prophet and Vector Autoregression. At that point in time the python packages for timeseries forecasting were not as good as R's. However, the python ecosystem for forecasting has grown a lot in the recent years. There are Python packages like [nitxla](https://nixtlaverse.nixtla.io/), [lightgbm](https://lightgbm.readthedocs.io/en/latest/Python-API.html), [catboost](https://catboost.ai/), as well as LLM forecasting models like [chronos](https://huggingface.co/amazon/chronos-t5-large) that offer many additional functionalities to leverage the use of new algorithms. 

### Data Characteristics

There are two dataset, one containing the historical volume data, another one containing the production data. In total there are:

- 624  inbound logistics Provider-Plant connections
- 18  plants
- 38  Providers

The historical transport volume data contains data since 2014-01-01 until 2022-10-01.
The historical production data contains data since 2014-01-01 until 2023-12-01. All data until October 2022 is actual produced values, the rest are planning values.

The two input data sources for this project are:

- **Inbound_Volume_Data.csv** contains the historical transported material volume since January 2014 until October 2022 on monthly basis. This data comes from the Logistics Parts Mangement System.
    - **Timestamp**: Monthly data of the format YYYY/MM.
    - **Provider**: Logistics Service Provider.
    - **Plant**: Assembly Plant. 
    - **Actual Vol [Kg]**: Actual transported volume from Provider to Plant in kg. 
    - **Expected Vol [Kg]**: Expected transported volume from Provider to Plant in kg.*
- **production_data.csv** contains the historical production levels of all the european plants in number of vehicles per month from January 2014 until October 2022. Data after October 2022 refers to planned production values. 
    - **Timestamp**:  Monthly data of the format YYYY/MM.
    - **Plant_X**: Column containing the production level for Plant X.

**Expected in this context means the volume value which the internal ERP system would calculate. That means, given the number of units in the call-off order and using the weights of the parts, the total expected weight of a delivery can be calculated. However, as mentioned before, due to the sync issue, the delivered volume and expected volume would differ.*

### Research question

The goal of this project is to improve the forecasting accuracy for the Inbound Logistics Volume of an International Automotive Company by creating a benchmark of statistical models, machine learning models, deep learning models and LLM Forecasting Models. 


The idea is to create a forecasting system which is accurate and robust to adapt for outliers and unexpected events. To evaluate the forecast accuracy the `MAE (Mean Absolute Error)` and `SMAPE (Symmetric Mean Absolute Error)` will be used. This will allow us to care about the fact that in some months the transportation volume could have been 0. 

The test timeframes are: 
- Jan 2022 - Apr 2022
- May 2022 - Aug 2022
- Jul 2022 - Oct 2022

This means that models tested in each frame can only be trained with data prior to that frame to avoid data leakage. 

One of the main Business KPI's to track forecast accuracy will be how many timeseries are in a particular `SMAPE` range, for that we will use the following intervals: 

- 0% to 10%
- 10 to 20%
- 20 to 30%
- 30 to 40%
- greater than 40%


The business experts are particularly interested in having a forecating systems for which most of the timeseries have a `SMAPE` of less or equal than 20%.

## Solution Approach

The approach to solve the problem is:

- (1) Analyze the data and create some graphs to explore the problem at hand. Create an Exploratory Data Analysis to explore Sesonality and Trends. 
- (2) Create a medallion architecture to track data quality: Bronze layer for raw data, Silver layer for cleaned data, Gold layer for aggregated or feature-engineered data. This is a common approach to model data in machine learning problems. Ref: [ Medallion Architecture](https://www.databricks.com/glossary/medallion-architecture) 
- (3) Train different models and store their accuracy results in the `parquet` format. We will train different models to cover the full range of current forecasting algorithms:
    - Statistical Models
    - Machine Learning Models
    - Deep Learning Models
    - LLM Timeseries Forecasting Models
- (4) Evaluate all the results, define the best performing models, and write up the lessons learned and possible next steps. 
- (5) Provide Business Timeseries Analysis Report, and Best Performing Models Report. 


### Overview of data

*Use the Pandas functions to provide an overview of the data set*


In [28]:
import pandas as pd
from datetime import datetime
import numpy as np

### Analysis Inbound Volume Data

In [14]:
df_vol = pd.read_csv("../data/raw/inbound_volume_data.csv", index_col=0)
df_vol["Year"] = df_vol["Timestamp"].apply(lambda x: x.split("/")[0]).astype(int)
df_vol["Month"] = df_vol["Timestamp"].apply(lambda x: x.split("/")[1]).astype(int)
df_vol["Timestamp"] = df_vol[["Year", "Month"]].apply(
    lambda x: datetime(x["Year"], x["Month"], 1), axis=1
)
# Create Timeseries Key
df_vol["ts_key"] = df_vol[["Provider", "Plant"]].apply(
    lambda x: x["Provider"] + "-" + x["Plant"], axis=1
)

In [15]:
df_vol.head()

,Timestamp,Provider,Plant,Actual Vol [Kg],Expected Vol [Kg],Year,Month,ts_key
0,2015-07-01,Provider_1,Plant_1,19096,20298,2015,7,Provider_1-Plant_1
1,2015-08-01,Provider_1,Plant_1,187139,189973,2015,8,Provider_1-Plant_1
2,2015-09-01,Provider_1,Plant_1,181149,187798,2015,9,Provider_1-Plant_1
3,2015-10-01,Provider_1,Plant_1,202677,202327,2015,10,Provider_1-Plant_1
4,2015-11-01,Provider_1,Plant_1,272902,271765,2015,11,Provider_1-Plant_1


In [16]:
df_vol.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47058 entries, 0 to 60518
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Timestamp          47058 non-null  datetime64[ns]
 1   Provider           47058 non-null  object        
 2   Plant              47058 non-null  object        
 3   Actual Vol [Kg]    47055 non-null  object        
 4   Expected Vol [Kg]  46951 non-null  object        
 5   Year               47058 non-null  int64         
 6   Month              47058 non-null  int64         
 7   ts_key             47058 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 3.2+ MB


In [40]:
print("The historical transport volume data contains data since", df_vol['Timestamp'].min(), " until ", df_vol['Timestamp'].max())
print("in Total it contains data for", df_vol['ts_key'].nunique(), " inbound logistics Provider-Plant connections")
print("in Total it contains data for", df_vol['Plant'].nunique(), " plants")
print("in Total it contains data for", df_vol['Provider'].nunique(), " Providers")
print("in Total it contains ", df_vol.shape[0], " rows.")
print("in Total it contains ", df_vol.shape[1], " columns.")

The historical transport volume data contains data since 2014-01-01 00:00:00  until  2022-10-01 00:00:00
in Total it contains data for 624  inbound logistics Provider-Plant connections
in Total it contains data for 18  plants
in Total it contains data for 38  Providers
in Total it contains  47058  rows.
in Total it contains  8  columns.


### Analysis Production Planning Data

In [ ]:
df_prod = pd.read_csv("../data/raw/production_data.csv", index_col=0)

df_prod.head()

,Timestamp,Plant_6,Plant_9,Plant_13,Plant_15,Plant_17,Plant_19,Plant_21,Plant_11,Plant_4,Plant_12,Plant_20,Plant_14,Plant_22,Plant_1,Plant_3,Plant_5,Plant_18,Plant_10
0,2014/01,11848,54138,33839,19438,6770,29098,45147,8578,326919,11668,93855,68067,13127,26363,26363,5799,13749,3633
1,2014/02,13528,53063,36690,24252,6943,26975,42656,9739,334540,11275,92865,66012,18565,29822,29822,7978,12880,2880
2,2014/03,12753,50551,38211,24978,7695,28991,46226,10619,356075,12306,99860,71251,19505,28915,28915,8149,13348,4165
3,2014/04,12681,52528,36979,23872,7436,27409,47929,10379,351736,11790,95409,66233,23983,29441,29441,5020,15650,4457
4,2014/05,13272,50855,31164,24145,7415,27959,47255,12155,332888,10407,97371,69890,21139,28110,28110,7087,15940,4035


In [24]:
# There was no production in April 2020 due to the Covid lockdown
df_prod[df_prod['Timestamp']=="2020/04"]

,Timestamp,Plant_6,Plant_9,Plant_13,Plant_15,Plant_17,Plant_19,Plant_21,Plant_11,Plant_4,Plant_12,Plant_20,Plant_14,Plant_22,Plant_1,Plant_3,Plant_5,Plant_18,Plant_10
75,2020/04,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
df_prod.describe()

,Plant_6,Plant_9,Plant_13,Plant_15,Plant_17,Plant_19,Plant_21,Plant_11,Plant_4,Plant_12,Plant_20,Plant_14,Plant_22,Plant_1,Plant_3,Plant_5,Plant_18,Plant_10
count,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.00000,120.000000,120.000000,120.000000,120.000000,120.000000
mean,13486.116667,41007.675000,24771.058333,22840.625000,7454.408333,28224.400000,42591.175000,14639.016667,308748.908333,7596.100000,88380.708333,83607.033333,67994.27500,68041.225000,68041.225000,11089.691667,11813.000000,4428.658333
std,2661.744876,11119.648605,6856.099049,6229.474383,1541.047944,7218.555478,8472.606887,3303.749060,64233.309356,3709.346588,33069.851139,19596.796614,29526.32378,15112.725171,15112.725171,3732.863437,3669.537809,1312.024989
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,11801.250000,33925.250000,18984.500000,18685.250000,6731.750000,24553.500000,38480.250000,13044.750000,282283.000000,4696.750000,61097.500000,72464.750000,39057.25000,62559.750000,62559.750000,8344.750000,9311.000000,3597.250000
50%,14181.500000,40437.500000,25675.500000,23967.000000,7801.000000,29025.500000,44473.500000,15307.500000,320305.000000,7987.500000,91953.000000,84206.000000,78820.50000,70230.500000,70230.500000,11745.000000,12699.500000,4820.000000
75%,15411.000000,49781.000000,30391.750000,27783.250000,8540.000000,33766.500000,48417.500000,16917.000000,353522.000000,10856.000000,113225.750000,98304.250000,90962.75000,78281.500000,78281.500000,14083.750000,14570.000000,5334.750000
max,18293.000000,65813.000000,38211.000000,33676.000000,9559.000000,41107.000000,57381.000000,18742.000000,409207.000000,13145.000000,153941.000000,121995.000000,113411.00000,90747.000000,90747.000000,16808.000000,17581.000000,7446.000000


The line above describes some statistics for the different plants. We can see that the min volume production is 0. Corresponding to the month of April 2020. The max production corresponds to the Plant_4 with a value of 409.207.

In [ ]:
print("In total it contains data for", len(np.unique(df_prod.describe().columns)), " plants. The same number as in the historical volume data")
print("In total it contains data for", len(np.unique(df_prod.describe().columns)), " plants. The same number as in the historical volume data")

in Total it contains data for 18  plants. The same number as in the historical volume data


In [35]:
# Convert the Timestamp to a Datetime Object
df_prod["Year"] = df_prod["Timestamp"].apply(lambda x: x.split("/")[0]).astype(int)
df_prod["Month"] = df_prod["Timestamp"].apply(lambda x: x.split("/")[1]).astype(int)
df_prod["Timestamp"] = df_prod[["Year", "Month"]].apply(
    lambda x: datetime(x["Year"], x["Month"], 1), axis=1
)

# Unpivot columns to rows
df_prod = pd.melt(
    df_prod.drop(columns=["Month", "Year"]),
    id_vars=["Timestamp"],
    var_name="Plant",
    value_name="Production",
)

In [37]:
df_prod.head()

,Timestamp,Plant,Production
0,2014-01-01,Plant_6,11848
1,2014-02-01,Plant_6,13528
2,2014-03-01,Plant_6,12753
3,2014-04-01,Plant_6,12681
4,2014-05-01,Plant_6,13272


In [39]:
print("The historical production data contains data since", df_prod['Timestamp'].min(), " until ", df_prod['Timestamp'].max())
print("in Total it contains ", df_prod.shape[0], " rows.")
print("in Total it contains ", df_prod.shape[1], " columns.")

The historical production data contains data since 2014-01-01 00:00:00  until  2023-12-01 00:00:00
in Total it contains  2160  rows.
in Total it contains  3  columns.


## Name of topic idea 2

### Data source

This data set is based on the recently hosted VN1 Forecasting Competition. Vandeput, Nicolas. (VN1 Forecasting - Accuracy Challenge. DataSource.ai) (https://www.datasource.ai/en/home/data-science-competitions-for-startups/phase-2-vn1-forecasting-accuracy-challenge/description)


### Data characterisitcs

There are two dataset, one containing the historical sales data, another one containing the historical prices data. In total there are:

- 15053 Client-Warehouse-Plant connections
- 46 Clients
- 328 Warehouses
- 11171 Products

The datasets contain data since 2020-07-06 until 2023-10-02.

- **VN1_Forecasting_Challenge_Sale.csv** contains historical sales data.
    - **Client**: Client ID.
    - **Warehouse**: Warehouse ID.
    - **Product**: Product ID. 
    - **YYYY-MM-DD**: Timestamp as column containing sales volume for that day. 
- **VN1_Forecasting_Challenge_Price.csv** contains historical price data. Not available in the future, therefore we need to take care of data leakeage in training. 
    - **Client**: Client ID.
    - **Warehouse**: Warehouse ID.
    - **Product**: Product ID. 
    - **YYYY-MM-DD**: Timestamp as column containing product price for that day. 
    
### Research question

*Describe a research question you're interested in answering using this data. State the response variable and possible predictor variables.*

### Overview of data

*Use the Pandas functions to provide an overview of the data set*


### Analysis Sales Data

In [61]:
df_sales = pd.read_csv("../data/raw/VN1_Forecasting_Challenge_Sales.csv")

In [62]:
df_sales.head()

,Client,Warehouse,Product,2020-07-06,2020-07-13,2020-07-20,2020-07-27,2020-08-03,2020-08-10,2020-08-17,...,2023-07-31,2023-08-07,2023-08-14,2023-08-21,2023-08-28,2023-09-04,2023-09-11,2023-09-18,2023-09-25,2023-10-02
0,0,1,367,7.0,7.0,7.0,7.0,7.0,7.0,7.0,...,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,7.0,0.0
1,0,1,639,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,5.0,5.0,6.0,5.0,1.0,2.0,2.0,18.0,2.0
2,0,1,655,21.0,21.0,21.0,25.0,35.0,35.0,35.0,...,9.0,4.0,2.0,9.0,8.0,6.0,0.0,17.0,21.0,37.0
3,0,1,1149,7.0,7.0,7.0,7.0,7.0,7.0,7.0,...,1.0,1.0,1.0,0.0,2.0,2.0,2.0,1.0,0.0,1.0
4,0,1,1485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,21.0,23.0,2.0,1.0,1.0,2.0,0.0,22.0,10.0,6.0


In [63]:
df_sales = pd.melt(df_sales, id_vars=['Client','Warehouse','Product'], var_name='Timestamp', value_name='Sale')
df_sales.sort_values(by=['Client', 'Warehouse', 'Product'], inplace = True)
df_sales.reset_index(drop=True, inplace=True)
df_sales['ts_key'] = (df_sales['Client'].astype(str) + "-" + 
                            df_sales['Warehouse'].astype(str)  + "-" +
                            df_sales['Product'].astype(str)) 

In [64]:
df_sales.head()

,Client,Warehouse,Product,Timestamp,Sale,ts_key
0,0,1,367,2020-07-06,7.0,0-1-367
1,0,1,367,2020-07-13,7.0,0-1-367
2,0,1,367,2020-07-20,7.0,0-1-367
3,0,1,367,2020-07-27,7.0,0-1-367
4,0,1,367,2020-08-03,7.0,0-1-367


In [66]:
print("The sales dataframe contains data since", df_sales['Timestamp'].min(), " until ", df_sales['Timestamp'].max())
print("in Total it contains data for", df_sales['ts_key'].nunique(), " Client-Warehouse-Plant connections")
print("in Total it contains data for", df_sales['Client'].nunique(), " Clients")
print("in Total it contains data for", df_sales['Warehouse'].nunique(), " Warehouses")
print("in Total it contains data for", df_sales['Product'].nunique(), " Products")
print("in Total it contains ", df_sales.shape[0], " rows.")
print("in Total it contains ", df_sales.shape[1], " columns.")

The sales dataframe contains data since 2020-07-06  until  2023-10-02
in Total it contains data for 15053  Client-Warehouse-Plant connections
in Total it contains data for 46  Clients
in Total it contains data for 328  Warehouses
in Total it contains data for 11171  Products
in Total it contains  2559010  rows.
in Total it contains  6  columns.


### Analysis Price Data

In [68]:
df_price = pd.read_csv("../data/raw/VN1_Forecasting_Challenge_Price.csv")

In [69]:
df_price.head()

,Client,Warehouse,Product,2020-07-06,2020-07-13,2020-07-20,2020-07-27,2020-08-03,2020-08-10,2020-08-17,...,2023-07-31,2023-08-07,2023-08-14,2023-08-21,2023-08-28,2023-09-04,2023-09-11,2023-09-18,2023-09-25,2023-10-02
0,0,1,367,10.900001,10.900001,10.900001,15.582857,27.289999,27.289999,27.289999,...,52.940000,NaN,52.940000,NaN,52.9400,51.860000,NaN,NaN,51.859997,NaN
1,0,1,639,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,106.900000,99.866000,106.897995,117.711670,113.4620,104.710000,121.850,118.425000,100.307220,103.450000
2,0,1,655,21.343332,21.343332,21.343332,22.881200,25.188000,25.188000,25.188000,...,30.794443,29.555000,28.605000,29.823334,31.4225,30.751665,NaN,30.691765,21.125238,7.572973
3,0,1,1149,11.480000,11.480000,11.480000,12.291429,14.320000,14.320000,14.320000,...,34.980000,34.980000,35.000000,NaN,34.9900,34.275000,34.275,34.280000,NaN,33.860000
4,0,1,1485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,29.936190,29.583042,29.765000,29.580000,29.5800,28.980000,NaN,28.977728,28.875000,28.628334


In [70]:
df_price = pd.melt(df_price, id_vars=['Client','Warehouse','Product'], var_name='Timestamp', value_name='Price')
df_price.sort_values(by=['Client', 'Warehouse', 'Product'], inplace = True)
df_price.reset_index(drop=True, inplace=True)
df_price['ts_key'] = (df_price['Client'].astype(str) + "-" + 
                            df_price['Warehouse'].astype(str)  + "-" +
                            df_price['Product'].astype(str)) 

In [71]:
print("The sales dataframe contains data since", df_price['Timestamp'].min(), " until ", df_price['Timestamp'].max())
print("in Total it contains data for", df_price['ts_key'].nunique(), " Client-Warehouse-Plant connections")
print("in Total it contains data for", df_price['Client'].nunique(), " Clients")
print("in Total it contains data for", df_price['Warehouse'].nunique(), " Warehouses")
print("in Total it contains data for", df_price['Product'].nunique(), " Products")
print("in Total it contains ", df_price.shape[0], " rows.")
print("in Total it contains ", df_price.shape[1], " columns.")

The sales dataframe contains data since 2020-07-06  until  2023-10-02
in Total it contains data for 15053  Client-Warehouse-Plant connections
in Total it contains data for 46  Clients
in Total it contains data for 328  Warehouses
in Total it contains data for 11171  Products
in Total it contains  2559010  rows.
in Total it contains  6  columns.
